7번 주제관련내용(콜센터난 고객센터에서의 음성인식을 통한 악성 고객 식별 및 차단 관련 AI서비스 개발)
문제점 
1. 콜센터 음성녹취록 데이터를 구하는 것이 불가능에 가까움(좀 더 찾아봐야함)
해결방향
1. 한국어 혐오표현 데이터 셋은 배포된 버전이 존재, 한국어 혐오표현에 대해 학습시킨 AI개발 후 클로버 API에서 음성을 텍스트로 변환해주는
   것을 활용하여 초기 프로토 타입을 제작한다.

한국어 혐오 데이터셋은 inmoonlight@github, warnikchow@github, beomi@github 님이 만드신 데이터입니다. 데이터 정보는 다음과 같습니다.
author: inmoonlight@github, warnikchow@github, beomi@github
repository: https://github.com/kocohub/korean-hate-speech
size:
train: 7,896 examples
dev: 471 examples
test: 974 examples
unlabeled: 2,033,893 examples
데이터 구조는 다음과 같습니다.
출처 : https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/korean_hate_speech.html

In [1]:
from Korpora import Korpora
Korpora.fetch("korean_hate_speech")

[korean hate speech] download unlabeled_comments_1.txt: 57.0MB [00:14, 3.85MB/s]                            
[korean hate speech] download unlabeled_comments_2.txt: 59.3MB [00:15, 3.81MB/s]                            
[korean hate speech] download unlabeled_comments_3.txt: 58.9MB [00:32, 1.80MB/s]                            
[korean hate speech] download unlabeled_comments_4.txt: 58.2MB [00:14, 3.94MB/s]                            
[korean hate speech] download unlabeled_comments_5.txt: 3.57MB [00:01, 2.74MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_1.txt: 44.7MB [00:15, 2.87MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_2.txt: 47.4MB [00:26, 1.76MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_3.txt: 48.5MB [00:23, 2.05MB/s]                            
[korean hate speech] download unlabeled_comments.news_title_4.txt: 47.9MB [00:12, 3.99MB/s]    

In [2]:
from Korpora import KoreanHateSpeechKorpus
corpus = KoreanHateSpeechKorpus()


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Authors :
        - Jihyung Moon* (inmoonlight@github)
        - Won Ik Cho* (warnikchow@github)
        - Junbum Lee (beomi@github)
        * equal contribution
    Repository : https://github.com/kocohub/korean-hate-speech
    References :
        - Moon, J., Cho, W. I., & Lee, J. (2020). BEEP! Korean Corpus of Online News
          Comments for Toxic Speech Detection. arXiv preprint arXiv:2005.12503.

    We provide the first human-annotated Korean corpus for toxic speech detection and the large unlabeled corpus.
    The data is comments from the Korean entertainment news aggregation platform.

    # License
    Creative Commons Attribution-ShareAlike 4.0 International License.
    Visit here for detail : https://creativec

In [31]:
corpus.train

KoreanHateSpeech.train: size=7896
  - KoreanHateSpeech.train.texts : list[str]
  - KoreanHateSpeech.train.titles : list[str]
  - KoreanHateSpeech.train.gender_biases : list[str]
  - KoreanHateSpeech.train.biases : list[str]
  - KoreanHateSpeech.train.hates : list[str]

In [36]:
import pandas as pd
import numpy as np

In [102]:
origin_train = pd.DataFrame(corpus.train)
origin_val = pd.DataFrame(corpus.dev)
origin_train.shape, origin_val.shape

((7896, 5), (471, 5))

In [103]:
origin_df = pd.concat([origin_train, origin_val])
origin_df.shape

(8367, 5)

In [104]:
# 혐오표현 정도 확인 / 적당하게 분포되어 있는 것을 확인
origin_df['hate'].value_counts()

none         3646
offensive    2688
hate         2033
Name: hate, dtype: int64

In [105]:
origin_df.reset_index(inplace = True, drop = True)

In [106]:
# 내용, 혐오발언 열만 남겨서 새로운 데이터 셋 선언
new_df = origin_df[['text', 'hate']]
new_df.shape

(8367, 2)

In [107]:
df_train = new_df[['text']]
df_label = new_df[['hate']]
df_train.shape, df_label.shape

((8367, 1), (8367, 1))

In [138]:
import re
re.sub('[-=.#/?:$}()]', '', df_train['text'][0])
df_train['text'] = df_train['text'].apply(lambda x: re.sub('[-=.#/?:$,}()~]', '', x))

<ipython-input-138-89b201086e93>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'] = df_train['text'].apply(lambda x: re.sub('[-=.#/?:$,}()~]', '', x))


In [140]:
df_train

,text
0,현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네
1,한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요ㅠ
2,못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...
3,12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데
4,1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노
...,...
8362,지현우 범죄 저지르지 않았나요
8363,여자인생 망칠 일 있나 ㅋㅋ
8364,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는
8365,할매젖x 뱃살x 몸매 s라인 유륜은 적당해야됨너무크거나 너무 작아도 x 엉덩이가 빵...


In [141]:
from konlpy.tag import Okt
okt = Okt()

In [153]:
df = pd.read_csv('C:\\Workspace\\python\\Data_Science\\dataA\\한국어불용어100.txt', encoding ='UTF-8', sep = '\s+', header = None)
# 품사를 무시하고 중복 배제
stopwords = set(df[0])
len(stopwords)

94

In [154]:
df_train['text'][0]

'현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네'

In [149]:
okt.morphs(df_train['text'][0], stem = True)

['현재',
 '호텔',
 '주인',
 '심정',
 '아',
 '18',
 '난',
 '마른하늘',
 '에',
 '날벼락',
 '맞다',
 '호텔',
 '망하다',
 '생기다',
 '누',
 '군',
 '계속',
 '추모',
 '받다']

In [157]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(okt.morphs(df_train['text'][0], stem = True)) 
print(tokenizer.word_index)

{'호텔': 1, '현재': 2, '주인': 3, '심정': 4, '아': 5, '18': 6, '난': 7, '마른하늘': 8, '에': 9, '날벼락': 10, '맞다': 11, '망하다': 12, '생기다': 13, '누': 14, '군': 15, '계속': 16, '추모': 17, '받다': 18}


In [108]:
# 범주형 데이터 원-핫 인코딩
# 출처 : 프로그래머스(https://programmers.co.kr/learn/courses/21/lessons/11044)
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data
dummy_columns = ['hate']
df_label_encoded = dummy_data(df_label, dummy_columns)

In [143]:
print(df_label_encoded.shape)
df_label_encoded

(8367, 3)


,hate_hate,hate_none,hate_offensive
0,1,0,0
1,0,1,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
8362,0,0,1
8363,1,0,0
8364,0,0,1
8365,1,0,0


In [110]:
from sklearn.model_selection import train_test_split
# 8:2로 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(df_train, df_label_encoded, test_size = 0.2,  stratify = df_label_encoded, random_state=1004)
# 훈련 데이터를 8:2로 검증데이터로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2,  stratify = y_train, random_state = 777)

In [115]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((5354, 1), (1339, 1), (1674, 1), (5354, 3), (1339, 3), (1674, 3))